In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [10]:
# 这个网络是LeNet 输入的是32*32的图像

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        # 这个网络输入的图像是 32*32维度的
        self.conv1 = nn.Conv2d(1, 6, 3)
        # 第一个卷积层的参数有6个 是卷积核参数
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 第二个卷积层有16个参数 也是卷积核参数
        # an affine operation: y = Wx + b
        # 经过两个卷积层后要开始全连接层

        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        # 第一个全连接层的的输入为16*6*6 因为上一个卷积层输出是6*6维度 16通道的特征图  这个FC有120个参数  因为这个FC输出为120个节点
        self.fc2 = nn.Linear(120, 84)
        # 第二个全连接层有84个参数
        self.fc3 = nn.Linear(84, 10)
        # 最后个FC有10个参数  说明网络是10分类的 对应的是10个手写数字的分类

    def forward(self, x):

        # 经过两次最大池化层 池化核都是2*2 步长为1的 并且没有补零的
        # 可以计算一下 32*32 经过3*3的卷积核 是30*30特征图,30*30经过2*2池化变成15*15的,15*15经过3*3卷积后是13*13,再2*2池化后是6*6的特征图
        # 所以说第一次的全连接层输入是16*6*6的  特征图被池化为6*6 然后有16个通道
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [12]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
# 输出网络的参数

10
torch.Size([6, 1, 3, 3])


In [ ]:
for i in params:
    print(i.shape)
# 循环把网络每一层的参数个数全部输出出来

In [5]:
# 让我们尝试一个32x32随机输入。注意：此网络（LeNet）的预期输入大小为32x32。要在MNIST数据集上使用此网络，请将图像从数据集中调整为32x32。
input = torch.randn(1,1,32,32)
# 随机生成数据给到网络 input相当于是 1个1通道的32*32的随机矩阵图作为图像
out = net(input)
# 把随机数据给到网络中去
out
#最后输出的是对应10个数字的概率

tensor([[-0.0863,  0.0486,  0.0403, -0.0589,  0.0419, -0.1323, -0.0768,  0.0404,
         -0.0025,  0.0558]], grad_fn=<AddmmBackward>)

In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))
# 在我们实际使用的时候 将具有随机梯度的所有参数和反向传播器的梯度缓冲区归零

In [7]:
# 损失函数采用一对（输出，目标）输入，并计算一个值，该值估计输出与目标之间的距离。

# nn软件包下有几种不同的 损失函数。一个简单的损失是：nn.MSELoss计算输入和目标之间的均方误差。

# 例如：
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1198, grad_fn=<MseLossBackward>)


现在，如果loss使用.grad_fn属性的属性向后移动， 您将看到如下所示的计算图：

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

In [9]:
# 因此，当我们调用时loss.backward()，整个图与损失是微分的，并且图中的所有张量都requires_grad=True 将具有.grad随梯度累积的张量。

# 为了说明，让我们向后走几步：
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [11]:
# 要反向传播错误，我们要做的就是loss.backward()。不过，您需要清除现有的渐变，否则渐变将累积到现有的渐变中。

# 现在，我们将调用loss.backward()，并查看向后前后conv1的偏差梯度。

#反向传播要先清零  因为torch中梯度是累加的
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
None


In [13]:
# 实践中使用的最简单的更新规则是随机梯度下降（SGD）：

# weight = weight - learning_rate * gradient
# 我们可以使用简单的Python代码实现此目的：

learning_rate = 0.01
# 做的操作是循环取出参数 然后用当前参数减去梯度值×学习率
for f in net.parameters():
    # 加下划线是自操作 也就是-=
    f.data.sub_(f.grad.data * learning_rate)

AttributeError: 'NoneType' object has no attribute 'data'

In [15]:
# 但是，当您使用神经网络时，您想要使用各种不同的更新规则，例如SGD，Nesterov-SGD，Adam，RMSProp等。为实现此目的，我们构建了一个小程序包：torch.optim实现所有这些方法。使用它非常简单：
import torch.optim as optim

# create your optimizer
# optimizer接受所有网络中的参数  设置学习率我0.01
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   #清零梯度 zero the gradient buffers
output = net(input)  # 再给一个输入 这个input就是我们随机生成的 1 1 32 32的随机输入
loss = criterion(output, target)
loss.backward() #backward会记录下梯度
optimizer.step()    # Does the update  
# 最后使用optimizer的step方法就会执行执行上面的那代码块中的操作 用随机梯度下降的方法 来更新weight权值